In [18]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re

%matplotlib inline 

In [ ]:
# В папке subsid (shared/homeworks/python_ds_miniprojects/5_subsid) 
# находятся файлы (tm_sales_1, tm_sales_2, ...) с продажами продуктов через телемаркетинг. 
# Каждый файл содержит, как минимум, 4 колонки (поля): FILIAL_ID, SUBS_ID, PROD_ID, ACT_DTTM.

In [4]:
path = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-o-bogdanova/shared/homeworks/python_ds_miniprojects/5_subsid'
os.listdir(path)

['subsid']

In [55]:
path_full = path+'/'+os.listdir(path)[0]
os.listdir(path_full)

['prod_activations_logs.csv',
 'tm_sales_1.csv',
 'tm_sales_2.csv',
 'tm_sales_3.csv']

In [176]:
all_tm = pd.DataFrame()
all_prod_logs = pd.DataFrame()
for i in os.listdir(path_full):
    if i.startswith('tm'): # choose tm-files
        df = pd.read_csv(path_full+'/'+i, sep = ';') \
            .dropna() \
            .reset_index(drop = True)#reading file #delete NaN
           
        for i in range(len(df['SUBS_ID'])): # если 'id' нет в начале SUBS_ID, то нужно его добавить
            if df.loc[i,'SUBS_ID'].startswith('id'):
                 continue
            else: 
                df.loc[i,'SUBS_ID'] = 'id' + df.loc[i,'SUBS_ID']
    
        list_columns = df.columns.tolist() # list of columns
        pattern = re.compile('DTTM')
    
        for i in list_columns: #find data columns
            if len(pattern.findall(i)) > 0:
                    df[i] = pd.to_datetime(df[i], dayfirst=True) # change to DateTime format
            else: continue
    
        df = df [["ACT_DTTM","FILIAL_ID","SUBS_ID","PROD_ID"]] #change column oder
        all_tm =all_tm.append(df)                     
    else:
        df_2 = pd.read_csv(path_full+'/'+i, sep = ';') \
            .dropna() \
            .reset_index(drop = True) #reading file #delete NaN
        
        list_columns = df_2.columns.tolist() # list of columns
        pattern = re.compile('DTTM')
    
        for i in list_columns: #find data columns
            if len(pattern.findall(i)) > 0:
                    df_2[i] = pd.to_datetime(df_2[i], dayfirst=True) # change to DateTime format
            else: continue
        all_prod_logs = all_prod_logs.append(df_2)


all_prod_logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00
1,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00
2,id7461794,3310,2020-03-20 17:25:00,2020-12-01 00:00:00
3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00
4,id8238421,1859,2020-03-01 11:42:00,2020-03-01 11:43:00
5,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00
6,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00
7,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00
8,id8741631,5677,2020-03-19 12:28:00,2020-12-01 00:00:00


In [177]:
 merge_data = all_tm\
     .merge(all_prod_logs, on = ['SUBS_ID', 'PROD_ID'])
merge_data

,ACT_DTTM,FILIAL_ID,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
0,2020-03-20 14:59:00,1,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00
1,2020-03-19 13:00:00,5,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00
2,2020-03-17 10:17:00,3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00
3,2020-03-16 16:28:00,2,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00
4,2020-03-15 14:21:00,6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00


In [184]:
# продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)

merge_data['difference'] = merge_data['END_DTTM'] - merge_data['START_DTTM']

merge_data=merge_data[merge_data.difference > '5m']
merge_data['SUBS_ID'].sort_values()

3    id2185490
0    id4651830
2    id5416547
4    id7642700
Name: SUBS_ID, dtype: object

In [182]:
# Сохранение в csv

data_to_csv = merge_data[['ACT_DTTM', 'FILIAL_ID', 'difference']].reset_index(drop = True)

# change date format 
for i in range(len(data_to_csv['ACT_DTTM'])):
    data_to_csv.loc[i, 'ACT_DTTM'] = data_to_csv.loc[i, 'ACT_DTTM'].strftime('%d-%m-%Y')
data_to_csv.to_csv('all_data.csv', sep = ';', index = False)
data_to_csv

,ACT_DTTM,FILIAL_ID,difference
0,20-03-2020,1,255 days 09:01:00
1,17-03-2020,3,8 days 00:43:00
2,16-03-2020,2,259 days 07:32:00
3,15-03-2020,6,0 days 09:21:00
